# Map visualization for Wuhan Coronavirus (2019-nCov) cases in Hong Kong

Download open data "List of buildings in which confirmed cases of novel coronavirus infection have stayed during the symptomatic phase in the past 14 days (English)" from Centre for Health Protection Hong Kong (CHP)

Other open data from CHP can be obtain from https://data.gov.hk/en-data/dataset/hk-dh-chpsebcddr-novel-infectious-agent

In [1]:
import requests
import io
import pandas as pd

response = requests.get('http://www.chp.gov.hk/files/misc/building_list_eng.csv')

file_object = io.StringIO(response.content.decode('utf-8'))
download_temp = pd.read_csv(file_object)
download_temp.to_csv('building_list_eng.csv')

In [2]:
# Read data from file
data = pd.read_csv("building_list_eng.csv") 

# Preview the first 5 lines of the loaded data 
data.head()

,Unnamed: 0,District,Building name,Date of the last case staying in the building during the symptomatic phase
0,0,Shatin,Alva Hotel by Royal,20/01/2020
1,1,Shatin,"Tower 1A, Oceanaire",21/01/2020
2,2,Shatin,"Tower 1, Lake Silver",23/01/2020
3,3,Tuen Mun,South Hillcrest,24/01/2020
4,4,Kwai Tsing,"Hong Mei House, Cheung Hong Estate",24/01/2020


In [3]:
# Changing columns name for easier operation
data.columns = ['ID','District','Building','Date']

In [4]:
# As source data doesn't have indication for building' corrisponding region/country,
# adding suffix for easier geo location search
data['Building'] = data['Building'].astype(str) + ', Hong Kong'

In [5]:
# Set column ID as Index
data = data.set_index('ID')

In [6]:
# Preview all data
data

,District,Building,Date
ID,,,
0,Shatin,"Alva Hotel by Royal, Hong Kong",20/01/2020
1,Shatin,"Tower 1A, Oceanaire, Hong Kong",21/01/2020
2,Shatin,"Tower 1, Lake Silver, Hong Kong",23/01/2020
3,Tuen Mun,"South Hillcrest, Hong Kong",24/01/2020
4,Kwai Tsing,"Hong Mei House, Cheung Hong Estate, Hong Kong",24/01/2020
5,Eastern,"Universal Towers, Hong Kong",25/01/2020
6,Sai Kung,"Tower 2, Tseung Kwan O Plaza, Hong Kong",26/01/2020
7,Yau Tsim Mong,"W Hong Kong, Hong Kong",28/01/2020
8,Central,"Four Seasons Hotel , Hong Kong",28/01/2020


In [7]:
# Install necessary libraries for map visualization
# !pip install geopandas
# !pip install geopy
# !pip install folium

In [8]:
# Import necessary libraries for map visualization
import geopandas as gpd
import geopy
from geopy import geocoders
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter

In [9]:
# Using address in Building column to search for geotag

# 1 - Function to delay between geocoding calls
g = geocoders.GoogleV3(api_key='PLEASE-ENTER-YOUR-OWN-GOOGLE-MAP-API-KEY')
geocode = RateLimiter(g.geocode, min_delay_seconds=5)
# 2- - Using address in Building column to search for geotag and append in column
data['Building'] = data['Building'].apply(geocode)
# 3 - Create GeoTag column which contain longitude, laatitude and altitude from Building column geotag result search 
data['GeoTag'] = data['Building'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - Split GeoTag column into latitude, longitude and altitude columns
data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(data['GeoTag'].tolist(), index=data.index)

In [10]:
# Preview all data again
data

,District,Building,Date,GeoTag,latitude,longitude,altitude
ID,,,,,,,
0,Shatin,"(1 Yuen Hong St, Sha Tin, Hong Kong, (22.38352...",20/01/2020,"(22.3835259, 114.2082408, 0.0)",22.383526,114.208241,0.0
1,Shatin,"(Oceanaire Tower 1a, 18 Po Tai St, Ma On Shan,...",21/01/2020,"(22.4151657, 114.2230053, 0.0)",22.415166,114.223005,0.0
2,Shatin,"(Lake Silver Tower 1, 599 Sai Sha Rd, Lake Sil...",23/01/2020,"(22.428605, 114.2430898, 0.0)",22.428605,114.243090,0.0
3,Tuen Mun,"(South Hillcrest, 3 Tuen Kwai Rd, Tuen Mun, Ho...",24/01/2020,"(22.412407, 113.9837095, 0.0)",22.412407,113.983710,0.0
4,Kwai Tsing,"(12 Ching Hong Rd, Tsing Yi, Hong Kong, (22.35...",24/01/2020,"(22.350358, 114.099329, 0.0)",22.350358,114.099329,0.0
5,Eastern,"(Universal Towers, 18-26 Kin Wah St, North Poi...",25/01/2020,"(22.2892024, 114.1973237, 0.0)",22.289202,114.197324,0.0
6,Sai Kung,"(Tseung Kwan O Plaza Tower 2, 1 Tong Tak St, T...",26/01/2020,"(22.3094024, 114.2625697, 0.0)",22.309402,114.262570,0.0
7,Yau Tsim Mong,"(1 Austin Road West Kowloon Station, Yau Ma Te...",28/01/2020,"(22.3049382, 114.160669, 0.0)",22.304938,114.160669,0.0
8,Central,"(8 Finance St, Central, Hong Kong, (22.2864877...",28/01/2020,"(22.2864877, 114.1569068, 0.0)",22.286488,114.156907,0.0


In [13]:
# Plotting map by folium
import folium

map1 = folium.Map(
    location=[22.28552, 114.15769],
    tiles='CartoDB dark_matter',
    zoom_start=11,
)

tooltip = 'Location'

data.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]],color='red', popup=row["Building"], tooltip=tooltip).add_to(map1), axis=1)
map1

In [14]:
# Save a map in html as well
map1.save("map1.html")

In [20]:
# Plotting map by folium plugin FastMarkerCluster to make data clustering visualization
from folium.plugins import FastMarkerCluster

map2 = folium.Map(location=[22.28552, 114.15769],
                        zoom_start=12,
                        tiles='CartoDB dark_matter')

FastMarkerCluster(data=list(zip(data['latitude'].values, data['longitude'].values))).add_to(map2)
folium.LayerControl().add_to(map2)

map2

In [21]:
# Save a map in html as well
map2.save("map2.html")

Quick checking GeoTag Helper

In [47]:
# Import necessary libraries for map visualization
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [48]:
# Using Geocoder "Nominatim OpenStreetMap"
locator = Nominatim(user_agent="GeoTag")
location = locator.geocode("Tower 1A, Oceanaire, Hong Kong")

In [49]:
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

AttributeError: 'NoneType' object has no attribute 'latitude'

In [60]:
# Using Geocoder "Google Map API"
from geopy import geocoders
g = geocoders.GoogleV3(api_key='PLEASE-ENTER-YOUR-OWN-GOOGLE-MAP-API-KEY')

#create an input address string
#you can also build this by reading from an input database and building a string

inputAddress = 'Tower 1A, Oceanaire, Hong Kong' 

#do the geocode
location = g.geocode(inputAddress, timeout=10)

#some things you can get from the result
print(location.latitude, location.longitude)

22.4151657 114.2230053
